In [ ]:
%pip install sentence-transformers

In [ ]:
%pip install seaborn

In [ ]:
from tinydb import TinyDB, Query

db = TinyDB('db.json')
table = db.table('articles')

articles = table.all()

print(f'loaded {len(articles)} articles')

In [ ]:
sentences = [x['abstract'] for x in articles if x['abstract'] != 'No abstract available.']

print(f'retaining {len(sentences)} articles')

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/allenai-specter')
embeddings = model.encode(sentences)

print(embeddings)

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import seaborn as sns

# Reduce embedding dimensionality using t-SNE for visualization
viz = TSNE(n_components=2, max_iter=10000).fit_transform(embeddings)
# viz_embeddings = TSNE(n_components=2, perplexity=100, n_iter=10000).fit_transform(embeddings)

# Plot
sns.scatterplot(x=viz[:, 0], y=viz[:, 1], s=100)
# sns.scatterplot(x=viz_embeddings[:, 0], y=viz_embeddings[:, 1], hue=labels, palette=palette, s=100)